In [ ]:
%load_ext dotenv
%dotenv
OPENAI_API_KEY = %env OPENAI_API_KEY
%env DSP_CACHEDIR=./cache/library
%env DSP_NOTEBOOK_CACHEDIR=./cache/notebook

> This will use the repo's cache to not need to call LLMs

# Importing DSPy and DSPy Inspector

In [ ]:
import dspy
from dspy_inspector import inspect

# Configuring the DSPy defaults

In [ ]:
lm = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=300)
rm = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(lm=lm, rm=rm)

# Creating a simple RAG program with a Classification subprogram

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="no more than a couple of sentences")

class CheckFaithfulness(dspy.Signature):
    """Verify that the answer is based on the provided context."""

    context = dspy.InputField(desc="Facts here are assumed to be true")
    answer = dspy.InputField()
    faithfulness = dspy.OutputField(desc="True/False indicating if answer is faithful to context")

class FaithfulnessClassifier(dspy.Module):
    def __init__(self):
        super().__init__()

        self.check_faithfulness = dspy.ChainOfThought(CheckFaithfulness)

    def forward(self, context, answer):
        faithfulness = self.check_faithfulness(context=context, answer=answer).faithfulness

        return dspy.Prediction(faithfulness=faithfulness)

class FaithfulRAG(dspy.Module):
    def __init__(self, passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.check_faithfulness = FaithfulnessClassifier()
    
    def forward(self, question):
        context = self.retrieve(question).passages
        answer = self.generate_answer(context=context, question=question).answer
        faithfulness = self.check_faithfulness(context=context, answer=answer).faithfulness

        return dspy.Prediction(context=context, answer=answer, faithfulness=faithfulness)

# Inspecting the uncompiled RAG program

> The created graph is bound to the created program's instance. Any subsequent calls to the program will update the graph.

Before any calls some information is empty, such as the parameters values.

In [ ]:
uncompiled_faithfulRAG = FaithfulRAG()
inspect(uncompiled_faithfulRAG)

After calling the program information is filled, check the parameters values.

In [ ]:
uncompiled_faithfulRAG("How many storeys are in the castle that David Gregory inherited?")

In [ ]:
uncompiled_faithfulRAG("When was the first FIFA World Cup held?")

In [ ]:
uncompiled_faithfulRAG("Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?")